# This notebook parses the attributed lives into patient panels by Organization 
##### Added Excel Automation including Filtering, Styling, Security, Pre-filtering 
#### Joe Eberle - 2/22/2022

In [1]:
# Import Libraries for Spreadsheet Automation 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import random
import math
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, GradientFill, Alignment, Color, Fill
from openpyxl import Workbook
import time

In [2]:
# Read in the CSV SQL output for Patient Attribution 
patient_panel = pd.read_excel('\\Users\josep\Documents\Kaleida\Attributed Patient Panel.xlsx')

In [3]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Provider_Summary = pd.read_excel('\\Users\josep\Documents\Kaleida\Organization_Provider_Summary.xlsx')

In [4]:
# Optional Verify the input file structures 
patient_panel.info()
#Organization_Provider_Summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149266 entries, 0 to 149265
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Organization_Class       147937 non-null  object        
 1   Organization_Name        148294 non-null  object        
 2   Provider_Name            149255 non-null  object        
 3   Patient_Name             149266 non-null  object        
 4   Birth_Date               149266 non-null  datetime64[ns]
 5   Age                      149266 non-null  int64         
 6   Gender                   148863 non-null  object        
 7   Phone                    148722 non-null  object        
 8   Deceased                 130700 non-null  float64       
 9   Deceased_Date            3305 non-null    object        
 10  Number_of_Conditions     143970 non-null  float64       
 11  MARA_Risk_Score          149251 non-null  float64       
 12  Number_of_HCC   

In [7]:
# simplify and Organize the patient Panels
patient_panel = patient_panel[['Provider_Name','Patient_Name', 'Birth_Date', 
                               'Gender', 'Phone','Is Patient cared for by Organization ?', 
                               'Is Patient attributed to correct PCP ?',
                               'Optional Comments', 'Deceased','Deceased_Date','Flagged for Test',
                               'Organization_Class', 'Organization_Name',
                               'Plan_Name', 'Payer_Name', 'Age_in_Years','Is Patient info Correct?',]]

KeyError: "['Is Patient cared for by Organization ?', 'Is Patient attributed to correct PCP ?'] not in index"

In [ ]:
# The following function creates patient panels based upon organization name 
def create_organization_patient_panel(org_name):
    Organization_panel = patient_panel[(patient_panel.Organization_Name == org_name)]    
    patient_rows = len(Organization_panel) - 1 
    if patient_rows > 2:
        spread_name = Organization_panel['Organization_Name'].iloc[0]
        if spread_name == '':
            spread_name = 'No_ORG_NAME_'  
        spread_name = spread_name.replace('/','')  
        lesser = min(patient_rows,25)
        for x in range(lesser):
            Organization_panel.iat[random.randint(0,patient_rows),10] = 'Yes'
        spath = '\\Users\josep\Documents\Kaleida\Validation\ '
        spread_name = spath + spread_name.replace(' ','_') + '.xlsx'
        spread_name = spread_name.replace(' ','')
        Organization_panel.to_excel(spread_name,sheet_name='Patient Attribution', index = False)  
        print(spread_name)
        return org_name

In [ ]:
Org_names = patient_panel.Organization_Name.unique()
for org in Org_names:
    create_organization_patient_panel(org)

In [ ]:
# Add the Provider Summary Excel Tab  
path = 'c:/Users/josep/Documents/Kaleida/Validation'
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/Validation/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/Validation/'
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
#Organization_panel = Organization_Provider_Summary[(Organization_Provider_Summary.Organization_Name == org_name)]  
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Save_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Organization_panel = Organization_Provider_Summary[(Organization_Provider_Summary.Organization_Name == org_name)]  
    Organization_panel = Organization_panel.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Organization_panel.to_excel(writer,sheet_name='Provider Summary', index = False)
    writer.save()
    writer.close()

In [ ]:
# The following function creates patient panels based upon organization name 
def create_sheet_one(work_book):
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment    
    import openpyxl
    
    ws1 = work_book.active
    ws1.title = "Patient Attribution"
    ws1.sheet_properties.tabColor = "1072BA"
    
    Physician Quality Scorecard
Practice Name: General Physician, PC
Provider: ALL
Last Updated: as of 3/31/2021

    ws1['A1'] = 'Physician Quality Scorecard'
    ws1['A2'] = 'Practice Name: General Physician, PC'
    ws1['A3'] = 'Provider: ALL'
    ws1['A4'] = 'Last Updated: as of 3/31/2021`' 
    ws1['A5'] = 'Remaining :'
    ws1['A6'] = '% of Validated as Accurate:'
    ws1['A6'] = 'Patient Population Criteria for Quality Metrics - ALL patients, regardless of insurance, who have' 
#had at least ONE visit in the past 3 years and have a GPPC PCP listed currently are pulled. 
Denominators are then stratified further based on measure criteria.
    
     
    for col in range(1,18):
        ws1[get_column_letter(col) + '1'].font = Font(bold=True ) 
    
    ws1["A1"].fill = PatternFill("solid", start_color="92D050")  
    ws1["B1"].fill = PatternFill("solid", start_color="92D050")  
    ws1["C1"].fill = PatternFill("solid", start_color="92D050")  
    ws1["D1"].fill = PatternFill("solid", start_color="92D050")  
    ws1["E1"].fill = PatternFill("solid", start_color="92D050")  
    ws1["F1"].fill = PatternFill("solid", start_color="F3FA6C")  
    ws1["G1"].fill = PatternFill("solid", start_color="F3FA6C")  
    ws1["H1"].fill = PatternFill("solid", start_color="F3FA6C") 
    ws1["I1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["J1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["K1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["L1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["M1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["N1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["O1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["P1"].fill = PatternFill("solid", start_color="D8DCD6")  
    ws1["Q1"].fill = PatternFill("solid", start_color="D8DCD6")  

# Work Sheet 1 - Patient Attribution - Turn Word Wrap On 
    for col in range(1,18):
        ws1[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)  
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws1.column_dimensions['A'].width = 30
    ws1.column_dimensions['B'].width = 30
    ws1.column_dimensions['C'].width = 12
    ws1.column_dimensions['D'].width = 10
    ws1.column_dimensions['E'].width = 15
    ws1.column_dimensions['F'].width = 12
    ws1.column_dimensions['G'].width = 15
    ws1.column_dimensions['H'].width = 20
    ws1.column_dimensions['I'].width = 10
    ws1.column_dimensions['J'].width = 15
    ws1.column_dimensions['K'].width = 10
    ws1.column_dimensions['L'].width = 25
    ws1.column_dimensions['M'].width = 25
    ws1.column_dimensions['N'].width = 25
    ws1.column_dimensions['O'].width = 25
    ws1.column_dimensions['P'].width = 25
    ws1.column_dimensions['Q'].width = 25    
    ws1.row_dimensions[1].height = 65   

# Replaces all underscores in header with two spaces 
    for col in range(1,18):
        ws1[get_column_letter(col) + '1'].value = ws1[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    import openpyxl
    from openpyxl.utils import get_column_letter

    FullRange = "A1:" + get_column_letter(ws1.max_column) \
    + str(ws1.max_row)
    ws1.auto_filter.ref = FullRange    

    from openpyxl.styles import NamedStyle, Font, Border, Side
    highlight = NamedStyle(name="highlight")
    highlight.font = Font(bold=True, size=15, color="92D050")
    return work_book  

In [ ]:
# The following function formats provider summary 
def create_sheet_two(work_book):
    from openpyxl.utils import get_column_letter
    from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment    
    import openpyxl
    
    sheets = work_book.sheetnames
   
    ws2 = work_book[sheets[1]]
   # ws = wb[sheets[n]]
    ws2.title = "Provider Summary"
    ws2.sheet_properties.tabColor = "1072BA"
     
    for col in range(1,7):
        ws2[get_column_letter(col) + '1'].font = Font(bold=True ) 
    
    ws2["A1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["B1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["C1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["D1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["E1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["F1"].fill = PatternFill("solid", start_color="92D050")  
    ws2["G1"].fill = PatternFill("solid", start_color="92D050")  

# Work Sheet 1 - Patient Attribution - Turn Word Wrap On 
    for col in range(1,6):
        ws2[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)  
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws2.column_dimensions['A'].width = 30
    ws2.column_dimensions['B'].width = 30
    ws2.column_dimensions['C'].width = 30
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 30
    ws2.column_dimensions['F'].width = 30

    ws2.row_dimensions[1].height = 60    

# Replaces all underscores in header with two spaces 
    for col in range(1,6):
        ws2[get_column_letter(col) + '1'].value = ws2[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    import openpyxl
    from openpyxl.utils import get_column_letter

    FullRange = "A1:" + get_column_letter(ws2.max_column) \
    + str(ws2.max_row)
    ws2.auto_filter.ref = FullRange    

    from openpyxl.styles import NamedStyle, Font, Border, Side
    highlight = NamedStyle(name="highlight")
    highlight.font = Font(bold=True, size=15, color="92D050")
    return work_book  

In [ ]:
def create_sheet_three(work_book):
    
    ws3 = work_book.create_sheet("Validation Summary")
    ws3.title = "Validation Summary" 
    ws3.sheet_properties.tabColor = "27AD1D"
    
    ws3.merge_cells('A1:F1')
    ws3.row_dimensions[1].height = 20
    top_left_cell = ws3['A1']
    top_left_cell.value = "Validation Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="ff0000")

    top_left_cell.border = Border(top=double, left=thin, right=thin, bottom=double)
    top_left_cell.fill = PatternFill("solid", fgColor="27AD1D")
    top_left_cell.fill = fill = GradientFill(stop=("27AD1D", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws3['A3'] = 'Number of Patients to Validate:'
    ws3['A4'] = 'Number of Patients Validate as Accurate:'
    ws3['A5'] = 'Number of Patients That are Not Validate:'
    ws3['A6'] = 'Total Validated :' 
    ws3['A7'] = 'Remaining :'
    ws3['A8'] = '% of Validated as Accurate:'

    ws3['A10'] = 'Total Distinct Providers:'
    ws3['A11'] = 'Total Patient Panel Size :'
    ws3.column_dimensions['A'].width = 50


    ws3['B3'] = '=COUNTIF(\'Patient Attribution\'!K2:K50000,"Yes")'
    ws3['B4'] = '=COUNTIF(\'Patient Attribution\'!F2:F50001,"Yes")'
    ws3['B5'] = '=COUNTIF(\'Patient Attribution\'!F2:F50002,"No")'
    ws3['B6'] = '=B4+B5'
    ws3['B7'] = '=B3-B6'
    ws3['B8'] = '=B4/B6'
    ws3['B8'].number_format = '0.0%'

    ws3['B11'] = '=COUNTA(\'Patient Attribution\'!B2:B50000)'
    return work_book 


In [ ]:
def create_sheet_four(work_book):
    
    ws4 = work_book.create_sheet("Instructions")
    ws4.title = "Instructions" 
    ws4.sheet_properties.tabColor = "FFFF00"
    
    ws4.merge_cells('A1:F1')
    ws4.row_dimensions[1].height = 20
    top_left_cell = ws4['A1']
    top_left_cell.value = "Instructions for Validation"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="ff0000")

    top_left_cell.border = Border(top=double, left=thin, right=thin, bottom=double)
    top_left_cell.fill = PatternFill("solid", fgColor="FFFF00")
    top_left_cell.fill = fill = GradientFill(stop=("FFFF00", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws4['A3'] = 'Please complete the validation for this spreadsheet following 5 instructions:'
    ws4['B4'] = 1
    ws4['B5'] = 2 
    ws4['B6'] = 3
    ws4['B7'] = 4
    ws4['B8'] = 5
    ws4['B9'] = 6
    ws4['B10'] = 7    
    ws4['B11'] = 8        
    ws4['C4'] = 'Please Filter column K  \'Flagged for Test\' to yes... it should select about 25 patients'
    ws4['C5'] = 'Validation needs to be completed on the pre-selected patients and returned to the sender you received them from by February 22, 2022.'
    ws4['C6'] = 'Please validate the data in the green columns on the Patient Panel spreadsheet.'
    ws4['C7'] = 'Enter a Y or N in the yellow column labeled \'Is Patient cared for by Organization ?\' '  
    ws4['C8'] = 'Enter a Y or N in the yellow column labeled \'Is Patient attributed to correct PCP ?\' ' 
    ws4['C9'] = 'If a NO is entered, indicate why in the yellow column labeled  \'Optional Comments \' '
    ws4['C10'] = 'Please review provider summary panel to ensure providers are part of organization. '     
    ws4['C11'] = 'After validation is complete add any new or missing providers on the new provider tab. '   
  
    
    ws4['C4'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['C5'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['C6'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['C7'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['C8'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B4'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B5'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B6'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B7'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B8'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws4['B9'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)    
    ws4['B10'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)   
    ws4['B11'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)       
    
    ws4.row_dimensions[4].height = 15
    ws4.row_dimensions[5].height = 15   
    ws4.row_dimensions[6].height = 15    
    ws4.row_dimensions[7].height = 15     
    ws4.row_dimensions[8].height = 15      
    ws4.row_dimensions[9].height = 15   
    ws4.row_dimensions[10].height = 15   

    ws4.column_dimensions['A'].width = 3
    ws4.column_dimensions['B'].width = 3
    ws4.column_dimensions['C'].width = 80
    
    return work_book  


In [ ]:
def create_sheet_five(work_book):
    
    ws5 = work_book.create_sheet("AAdding new Physicians")
    ws5.title = "Adding new Physicians" 
    ws5.sheet_properties.tabColor = "DA9694"
    
    ws5.merge_cells('A1:G1')
    ws5.row_dimensions[1].height = 20
    top_left_cell = ws5['A1']
    top_left_cell.value = "Adding, Updating, or Removing Physicians"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="ff0000")

    top_left_cell.border = Border(top=double, left=thin, right=thin, bottom=double)
    top_left_cell.fill = PatternFill("solid", fgColor="DA9694")
    top_left_cell.fill = fill = GradientFill(stop=("DA9694", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws5['A2'] = 'Provider Name'
    ws5['B2'] = 'Provider NPI'
    ws5['C2'] = 'Provider Credentials'
    ws5['D2'] = 'Is Provider a Primary Care Physician PCP (Y/N) ?'
    ws5['E2'] = 'Is Provider a Specialist (Y/N) ? '
    ws5['F2'] = 'If Specialist enter Provider Specialty'
    ws5['G2'] = 'Does the Provider need to be added or removed or Updated (A/R/U) ? '    
    ws5['A2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['B2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['C2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['D2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['E2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['F2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    ws5['G2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    
    for col in range(1,8):
        ws5[get_column_letter(col) + '2'].font = Font(bold=True ) 
        ws5[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws5[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor="92D050")
    
    ws5.column_dimensions['A'].width = 30
    ws5.column_dimensions['B'].width = 20
    ws5.column_dimensions['C'].width = 10
    ws5.column_dimensions['D'].width = 20
    ws5.column_dimensions['E'].width = 20
    ws5.column_dimensions['F'].width = 20   
    ws5.column_dimensions['G'].width = 20       
    
    ws5.row_dimensions[2].height = 60
 
   
    return work_book  


In [ ]:
# Automate the worksheet 
def create_work_book(work_book):
    create_sheet_one(wb)
    create_sheet_two(wb)
    create_sheet_three(wb)
    create_sheet_four(wb)
    create_sheet_five(wb)    
    return work_book      

In [ ]:
path = 'c:/Users/josep/Documents/Kaleida/Validation'
Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/Validation/'
Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/Validation2/'
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
extension = 'xlsx'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))

# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'

for x in range(len(files)): 
    print(files[x]) 
    wb = load_workbook(Path_to_Excel + files[x])
    org_name =  files[x].replace('.xlsx','')
    #print (org_name)
    create_work_book(wb)
    #time.sleep(.5)
    wb.save(Path_to_Save_Excel + files[x]) 
#   print(files[x]) 

In [ ]:
# Optional - Print out the compressive List of Patient Panels 
#Org_names = patient_panel.Organization_Name.unique()
#for org in Org_names:
#        print(org)

In [ ]:
# patient_panel.describe()

In [ ]:
# test examples 
#create_organization_patient_panel("GPPC PCP Springville South Cascade")
#create_organization_patient_panel("EC PCP Erie County Medical Center FM Clinic")


In [ ]:
#Provider_panel = patient_panel[(patient_panel.Provider_Name == "ADAMS, TIMOTHY")] 

In [ ]:
#   create_work_book(wb)    

In [ ]:
# patient_panel['Gender'].value_counts()

In [ ]:
# patient_panel['Organization_Name'].value_counts()

In [ ]:
# Organization_panel.head()

In [ ]:
#wb.security.workbookPassword = 'population health'
#wb.security.lockStructure = True

In [ ]:
#spread_name = '\\Users\josep\Documents\Kaleida\'+ Organization_panel
#Organization_panel.to_excel('\\Users\josep\Documents\Kaleida\GPPC PCP Springville South Cascade.xlsx')
#"C:\Users\josep\Documents\Kaleida\Validation\AA_PCP_Alan_M_Antfleck_Physician_PLLC_Delaware_Ave.xlsx"